In [1]:
import copy
import json
import os
import warnings
from typing import Any, Dict, List

import torch
import torch.distributed as dist
from pytorch_lightning import LightningModule
from pytorch_lightning.utilities import rank_zero_only

from nanodet.data.batch_process import stack_batch_img
from nanodet.optim import build_optimizer
from nanodet.util import convert_avg_params, gather_results, mkdir

from nanodet.model.arch import build_model
from nanodet.model.weight_averager import build_weight_averager

c:\Users\franc\anaconda3\envs\nanodet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
import os
import warnings
import yaml

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import TQDMProgressBar
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from nanodet.data.collate import naive_collate
from nanodet.data.dataset import build_dataset
from nanodet.evaluator import build_evaluator
from nanodet.trainer.task import TrainingTask
from torchvision.transforms import ToTensor, ToPILImage
from nanodet.util import (
    NanoDetLightningLogger,
    cfg,
    convert_old_model,
    env_utils,
    load_config,
    load_model_weight,
    mkdir,
)

In [3]:
load_config(cfg, 'cfg/task' + str(0) + '.yml')

In [4]:
print(cfg)

class_names: ['aereoplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person']
data:
  train:
    ann_path: C:/Dataset/VOC/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/annotations
    class_names: ['aereoplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person']
    img_path: C:/Dataset/VOC/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages
    input_size: [320, 320]
    keep_ratio: True
    name: XMLDataset
    pipeline:
      brightness: 0.2
      contrast: [0.8, 1.2]
      flip: 0.5
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
      perspective: 0.0
      rotation: 0
      saturation: [0.8, 1.2]
      scale: [0.6, 1.4]
      shear: 0
      stretch: [[1, 1], [1, 1]]
      translate: 0.2
  val:
    ann_path: C:/Dataset/VOC/VOCtest_06-Nov-2007/VOCdevkit/VOC2007/annotations
    class_names: ['aereoplane', 'bicyc

In [5]:
model = build_model(cfg.model)

model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


In [6]:
print(model)

NanoDetPlus(
  (backbone): ShuffleNetV2(
    (conv1): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (stage2): Sequential(
      (0): ShuffleV2Block(
        (branch1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): LeakyReLU(negative_slope=0.1, inplace=True)
        )
        (branch2): Sequential(
          (0): Conv2d(24, 58, kernel_size=(1, 1),

In [7]:
print(sum(p.numel() for p in model.parameters()))

4193299
